<a href="https://colab.research.google.com/github/woghd8503/kernel-academy-web-crawling/blob/main/bsoup_hard(%ED%95%99%EC%83%9D%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹 데이터 수집 고급 심화: "Books to Scrape" 사이트 정복하기 📚🐛 (고급)

안녕하세요! 이전 과정들을 통해 웹 스크래핑의 기초와 중급 기술들을 익혔습니다. 이번 고급 심화 과정에서는 스크래핑 연습용으로 만들어진 "Books to Scrape" ([http://books.toscrape.com/](http://books.toscrape.com/)) 사이트를 대상으로, 보다 복잡하고 실제적인 스크래핑 작업을 수행해 보겠습니다.

**학습 목표**
1. 대상 웹사이트의 구조를 분석하고 스크래핑 전략 수립하기.
2. 여러 계층(카테고리 목록 -> 책 목록 -> 책 상세)에 걸쳐 데이터 수집하기.
3. 페이지네이션(Pagination) 처리, 데이터 클리닝, 상세 오류 처리 등 고급 기법 적용하기.
4. 스크래핑 작업을 위한 함수를 모듈화하여 코드의 재사용성 및 가독성 높이기.
5. 수집된 데이터를 구조화하고 파일로 저장하기.

**준수 사항:** 웹 스크래핑 시에는 항상 해당 웹사이트의 `robots.txt` 파일과 이용 약관을 확인하고, 서버에 과도한 부담을 주지 않도록 주의해야 합니다. "Books to Scrape"는 학습 목적으로 자유로운 스크래핑이 허용된 사이트입니다.

---

## 📄 1단계: 사이트 분석 및 환경 설정

스크래핑을 시작하기 전에 대상 웹사이트인 "Books to Scrape"의 구조를 파악하고, 필요한 라이브러리를 준비합니다.

**1.1. `robots.txt` 확인**
웹사이트의 루트 경로에 `/robots.txt`를 붙여 접속하면 (예: `http://books.toscrape.com/robots.txt`) 해당 파일의 내용을 확인할 수 있습니다. "Books to Scrape"의 `robots.txt`는 모든 유저 에이전트(`User-agent: *`)에 대해 모든 경로의 접근을 허용(`Disallow:` 항목 없음)하고 있습니다. 이는 학습용 사이트이므로 매우 관대한 정책입니다.

**1.2. 사이트 구조 파악**
* **메인 페이지**: 카테고리 목록(사이드바), 책 목록 표시.
* **카테고리 페이지**: 특정 카테고리에 속한 책 목록 표시, 페이지네이션 존재.
* **책 상세 페이지**: 개별 책의 상세 정보 (제목, 가격, 재고, 설명, UPC, 별점 등) 표시.

**1.3. 필요한 라이브러리 임포트 및 기본 설정**

In [ ]:
# !pip install requests beautifulsoup4 pandas # Colab에 이미 설치되어 있다면 실행 불필요
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin # 상대 URL을 절대 URL로 변환
import time # 요청 간 지연시간 설정
import re # 정규 표현식 사용
import random # User-Agent 랜덤 선택용 (선택 사항)

In [ ]:
# 기본 URL 설정
BASE_URL = 'http://books.toscrape.com/'

In [ ]:
CATALOGUE_URL = urljoin(BASE_URL, 'catalogue/') # 책 상세페이지 URL 조립 시 사용

In [ ]:
# User-Agent 설정 (다양한 User-Agent를 준비하여 랜덤으로 사용하는 것도 좋은 방법)
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
]
headers = {'User-Agent': random.choice(USER_AGENTS)}

In [ ]:
print(f"스크래핑 기본 URL: {BASE_URL}")
print(f"사용될 User-Agent: {headers['User-Agent']}")

스크래핑 기본 URL: http://books.toscrape.com/
사용될 User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36


**1.4. 웹 페이지 요청 함수 정의**

안정적인 스크래핑을 위해 `User-Agent` 설정, 요청 간 지연, 기본적인 오류 처리를 포함하는 웹 페이지 요청 함수를 정의합니다.

In [ ]:
time.sleep(10)

In [ ]:
def get_soup(url, retries=3, delay=1):
    """지정된 URL로부터 BeautifulSoup 객체를 반환하는 함수"""
    for i in range(retries):
        try:
            print(f"페이지 요청 중 ({i+1}번째 시도): {url}")
            time.sleep(random.uniform(delay, delay + 1)) # delay가 만약 1이라면, 1~2초사이 기다렸다가 실행
            response = requests.get(url, headers=headers, timeout=15) # timeout 시간만큼 기다린다
            response.raise_for_status() # HTTP 오류 발생 시 [400,404,500~] 그냥 진행하지말고  파이썬 에러 발생
            return BeautifulSoup(response.content, 'html.parser')

        except requests.exceptions.Timeout:
            print(f"타임아웃 발생: {url}")

        except requests.exceptions.HTTPError as e: # HTTPError 예외 타입
            print(f"HTTP 오류 발생 ({url}): {e.status_code} {e.response.reason}")
            if e.response.status_code == 404: # 404
                print("404 Not Found. 해당 URL은 존재하지 않습니다.")
                return None # 404 에러 시 재시도 불필요

        except requests.exceptions.RequestException as e:
            print(f"요청 중 오류 발생 ({url}): {e}")

        if i < retries - 1: # 마지막 시도가 아니면 대기 후 재시도
            print(f"{delay * (i+1)}초 후 재시도...")
            time.sleep(delay * (i+1)) # time.sleep
        else:
            print(f"최대 재시도 횟수({retries}) 초과. {url} 요청 실패.")
    return None

In [ ]:
# 함수 테스트 (메인 페이지)
main_soup = get_soup(BASE_URL) # get_soup

페이지 요청 중 (1번째 시도): http://books.toscrape.com/


In [ ]:
if main_soup:
    print(f"\n메인 페이지 제목: {main_soup.title.string.strip()}")
else:
    print("\n메인 페이지 로드 실패.")


메인 페이지 제목: All products | Books to Scrape - Sandbox


## 📚 2단계: 카테고리 및 책 URL 수집

이제 사이트의 모든 카테고리 링크를 수집하고, 각 카테고리 페이지를 탐색하여 개별 책들의 상세 페이지 URL을 수집합니다.

**2.1. 모든 카테고리 URL 수집**

메인 페이지의 사이드바에는 책 카테고리 목록이 있습니다. 각 카테고리명과 해당 카테고리 페이지로 연결되는 URL을 추출합니다.

In [ ]:
def get_category_urls(soup):
    """메인 페이지 BeautifulSoup 객체에서 모든 카테고리 이름과 URL을 추출하는 함수"""
    category_links = []
    if not soup:
        return category_links

    # 사이드바의 카테고리 목록은 ul.nav-list > li > ul > li > a 형태
    # 좀 더 구체적으로는 div.side_categories > ul.nav-list > li > ul 카테고리 리스트 최종 태그가 존재 하는곳
    side_categories_ul = soup.select_one('div.side_categories > ul.nav-list > li > ul') # CSS 선택자
    if side_categories_ul:
        for li_tag in side_categories_ul.find_all('li'):
            a_tag = li_tag.find('a')
            if a_tag and a_tag.has_attr('href'):
                category_name = a_tag.get_text(strip=True)
                category_relative_url = a_tag['href']
                category_full_url = urljoin(BASE_URL, category_relative_url)
                category_links.append({'name': category_name, 'url': category_full_url})
    return category_links

In [ ]:
if main_soup:
    all_categories = get_category_urls(main_soup) # get_category_urls
    print(f"\n총 {len(all_categories)}개의 카테고리를 찾았습니다.")
    for cat in all_categories[:3]: # 처음 3개 카테고리 정보 출력
        print(cat)
else:
    all_categories = []
    print("\n메인 페이지 soup 객체가 없어 카테고리를 찾을 수 없습니다.")


총 50개의 카테고리를 찾았습니다.
{'name': 'Travel', 'url': 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'}
{'name': 'Mystery', 'url': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'}
{'name': 'Historical Fiction', 'url': 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'}


**2.2. 특정 카테고리 내 모든 책 URL 수집 (페이지네이션 처리 포함)**

카테고리 페이지에는 여러 권의 책이 나열되며, 책이 많을 경우 여러 페이지에 걸쳐 표시됩니다 ('Next' 버튼 존재).
하나의 카테고리 URL을 입력받아, 해당 카테고리의 모든 페이지를 방문하며 각 책의 상세 페이지 URL을 수집하는 함수를 작성합니다.
여기서는 학습을 위해 각 카테고리당 최대 페이지 수를 제한할 수 있습니다.

In [ ]:
def get_book_urls_from_category(category_url, max_pages_per_category=5):
    """특정 카테고리 페이지(및 하위 페이지)에서 모든 책의 상세 URL을 수집하는 함수"""
    book_urls = set() # 중복 방지를 위해 set 사용
    current_page_url = category_url
    pages_crawled = 0

    while current_page_url and pages_crawled < max_pages_per_category:
        category_soup = get_soup(current_page_url) # get_soup
        pages_crawled += 1

        if not category_soup:
            print(f"카테고리 페이지 로드 실패: {current_page_url}")
            break

        # article.product_pod > div.image_container > a > img
        # or article.product_pod > h3 > a 두개가 존재
        for book_pod in category_soup.select('article.product_pod'):
            book_link_tag = book_pod.select_one('h3 > a')
            if book_link_tag and book_link_tag.has_attr('href'): # book link가 있고 그 링크에 하이퍼 링크가 있느냐?(href)
                # 앞에 상대 경로는 current ~ 안에 다 있음
                book_relative_url = book_link_tag['href'].replace('../', '') # 경로 조정
                book_full_url = urljoin(CATALOGUE_URL, book_relative_url)
                book_urls.add(book_full_url) # list의 append와 동일



        # 다음 페이지(Next) 버튼 찾기
        next_button = category_soup.select_one('li.next > a') # CSS 선택자 for next button
        if next_button and next_button.has_attr('href'):
            next_page_relative_url = next_button['href']
            current_page_url = urljoin(current_page_url, next_page_relative_url)
        else:
            print(f'{category_url} 카테고리의 마지막 페이지에 도달했거나, 다음 페이지 버튼이 없습니다!')
            current_page_url = None #반복 종료

    return list(book_urls)

In [ ]:
# 테스트: 첫 번째 카테고리에 대해서만 책 URL 수집 (최대 2페이지만)
all_book_detail_urls = set()
all_categories

[{'name': 'Travel',
  'url': 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'},
 {'name': 'Mystery',
  'url': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'},
 {'name': 'Historical Fiction',
  'url': 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'},
 {'name': 'Sequential Art',
  'url': 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html'},
 {'name': 'Classics',
  'url': 'http://books.toscrape.com/catalogue/category/books/classics_6/index.html'},
 {'name': 'Philosophy',
  'url': 'http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html'},
 {'name': 'Romance',
  'url': 'http://books.toscrape.com/catalogue/category/books/romance_8/index.html'},
 {'name': 'Womens Fiction',
  'url': 'http://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html'},
 {'name': 'Fiction',
  'url': 'http://books.toscrape.com/catalogue/category/books/fiction_10/ind

In [ ]:
if all_categories:
    #sample_category = all_categories[0] # travel 카테고리
    sample_category = all_categories[1] # mystery 카테고리
    print(f'{sample_category} 카테고리에서 정보 수집중')
    book_url_in_sample_category = get_book_urls_from_category(sample_category['url'],max_pages_per_category=2)
    for url in book_url_in_sample_category:
        print(url)
else:
    print("카테고리 정보가 없어 책 URL을 수집할 수 없습니다.")

{'name': 'Mystery', 'url': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'} 카테고리에서 정보 수집중
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/category/books/mystery_3/index.html
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/category/books/mystery_3/page-2.html
http://books.toscrape.com/catalogue/category/books/mystery_3/index.html 카테고리의 마지막 페이지에 도달했거나, 다음 페이지 버튼이 없습니다!
http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html
http://books.toscrape.com/catalogue/1st-to-die-womens-murder-club-1_2/index.html
http://books.toscrape.com/catalogue/the-mysterious-affair-at-styles-hercule-poirot-1_452/index.html
http://books.toscrape.com/catalogue/the-girl-in-the-ice-dci-erika-foster-1_65/index.html
http://books.toscrape.com/catalogue/the-silkworm-cormoran-strike-2_280/index.html
http://books.toscrape.com/catalogue/blood-defense-samantha-brinkman-1_8/index.html
http://books.toscrape.com/catalogue/hide-away-eve-duncan-20_620/ind

## 📖 3단계: 개별 책 상세 정보 스크래핑

수집된 각 책의 상세 페이지 URL로 접속하여 책의 제목, 가격, 재고 상태, 상품 설명, UPC, 별점 등 다양한 정보를 추출합니다.

**3.1. 책 상세 정보 추출 함수 정의**

이 함수는 하나의 책 상세 페이지 URL을 입력받아 해당 페이지의 모든 필요한 정보를 추출하여 딕셔너리 형태로 반환합니다.
정보가 누락된 경우를 대비해 오류 처리 및 기본값 설정을 포함합니다.

In [ ]:
def scrape_book_details(book_url):
    """개별 책 상세 페이지에서 정보를 추출하는 함수"""
    book_data = {'url': book_url} # 기본 정보로 URL 추가
    soup = get_soup(book_url) # get_soup

    if not soup:
        print(f"책 상세 정보 로드 실패: {book_url}")
        book_data['title'] = "정보 없음 (로드 실패)"
        return book_data

    # 제목 (h1 태그)
    title_tag = soup.select_one('div.product_main > h1') # CSS 선택자 for title
    book_data['title'] = title_tag.get_text(strip=True) if title_tag else None # get_text

    # 가격 (p.price_color)
    price_tag = soup.select_one('div.product_main > p.price_color') # CSS 선택자 for price
    if price_tag:
        price_tag = price_tag.get_text(strip=True) # 빈칸 제거 하기
        book_data['price_raw'] = price_tag
        try: # £ 말고 다른 단위가 있을수 있으니,
            book_data['price'] = float(price_tag.replace('£', ''))
        except ValueError:
            book_data['price'] = None
            print(f'가격 변환 오류 {price_tag}')
    else:
        book_data['price_raw'] = None
        book_data['price'] =None


    # 재고 상태 (p.instock.availability)
    stock_tag = soup.select_one('div.product_main > p.instock.availability') # CSS 선택자 for stock
    if stock_tag:
        stock_text = stock_tag.get_text(strip=True) # 앞뒤 공백 제거
        book_data['stock_availability_raw'] = stock_text
        # In stock (14 available) -> 숫자만 추출 하려면 어떻게 해야할까?
        # regex 정규표현식을 사용하자
        # 괄호를 찾고 () 거기에서 '숫자' 만 찾는다
        match = re.search(r'\((\d+) available\)', stock_text)
        # \(\) 괄호로 시작(해서 괄호로 끝나는 부분)을 찾아라
        # \d+ 숫자
        if match:
            book_data['stock_available_count'] = int(match.group(1)) # 매치된 데이터 한개 들고 와라
        else:
            book_data['stock_available_count'] = 0
    else:
        book_data['stock_availability_raw']= None
        book_data['stock_available_count'] = 0


    # 별점 (p.star-rating 에 붙은 클래스 이름으로 확인. 예: "star-rating Four")
    # 별점은 숫자로 바꿔서 저장하기 !
    star_rating_tag = soup.select_one('div.product_main > p.star-rating') # CSS 선택자 for star rating
    if star_rating_tag and star_rating_tag.has_attr('class'): # 'class'
        rating_classes = star_rating_tag['class']
        rating_map = {'One' :1, 'Two': 2, 'Three':3, 'Four':4, 'Five':5}
        book_data['star_rating'] = None
        for r_class in rating_classes:
            if r_class in rating_map:
                book_data['star_rating'] = rating_map[r_class]
                break

    else:
        book_data['star_rating'] = None

    # 상품 설명 (div#product_description + p)
    desc_header = soup.find('div', id='product_description') # find, 'product_description'
    if desc_header and desc_header.find_next_sibling('p'): # find_next_sibling
        book_data['description'] = desc_header.find_next_sibling('p').get_text(strip=True) # get_text
        # prodcut_description 다음에 등장하는 p태그를 찾아라, 왜? -> 지금까지는 product_description안에 존재하는 특정 태그를 찾는 방식이였지만,
        # 지금은 그 다음에 등장하는 p태그를 찾는게 목표다 왜 단독 으로 안찾냐? -> p태그는 너무 범용적으로 쓰이는 태그

    else:
        book_data['description'] = None

    # 상품 정보 테이블 (UPC, Product Type, Price (excl. tax), etc.)
    product_info_table = soup.find('table', class_='table-striped') # find, 'table-striped'
    if product_info_table:
        for row in product_info_table.find_all('tr'):
            header = row.find('th').get_text(strip=True) if row.find('th') else None # get_text
            value = row.find('td').get_text(strip=True) if row.find('td') else None # get_text
            if header and value:
                # 딕셔너리 구성해보자, 헤더 이름을 키로 사용, (공백과 특수문자도 제거해서) Regex를 사용!
                # 여기부터 이어서 진행
                # 안녕!!! 안녕하세요? ^^ -> regex !, # , 안^녕^? , 아!..배가..고,,프다..#강남#맛집#오늘의~~  -> regex 정규표현식
                clean_header = re.sub(r'[^a-zA-Z0-9_]', '',header.lower().replace(' ', '_')) # re.sub, regex pattern for cleaning header
                # 문자 단위 기준으로 생각하기, a부터 z, A부터 Z, 0부터 9 그리고 _ 까지는 제외하고(^) 나머지 문자들은 전부 제거
                book_data[clean_header] = value
                if header == 'UPC': book_data['upc'] = value # 'UPC'
                if header == 'Availability': book_data['stock_from_table'] = value
                if header == 'Number of reviews': book_data['num_reviews'] = int(value) if value.isdigit() else 0
    return book_data

In [ ]:
# 함수 테스트 (이전에 수집한 책 URL 중 하나 사용)
if book_url_in_sample_category:
    sample_book_url = list(book_url_in_sample_category)[0] # index (e.g. 0)
sample_book_url

In [ ]:
print(f"\n샘플 책 상세 정보 스크래핑: {sample_book_url}")
detailed_book_info = scrape_book_details(sample_book_url) # scrape_book_details


샘플 책 상세 정보 스크래핑: http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html


In [ ]:
if detailed_book_info:
    for key, value in detailed_book_info.items():
        if key == 'description' and value and len(value) > 70:
            print(f"{key}: {value[:70]}...")
        else:
            print(f"{key}: {value}")
else:
    print("\n스크래핑할 책 URL이 없습니다.")

url: http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html
title: A Study in Scarlet (Sherlock Holmes #1)
price_raw: £16.73
price: 16.73
stock_availability_raw: In stock (14 available)
stock_available_count: 14
star_rating: 2
description: In the debut of literature's most famous sleuth, a dead man is discove...
upc: 63ee5bc46066a8a8
product_type: Books
price_excl_tax: £16.73
price_incl_tax: £16.73
tax: £0.00
availability: In stock (14 available)
stock_from_table: In stock (14 available)
number_of_reviews: 0
num_reviews: 0


**3.2. 수집된 모든 책 URL에 대해 상세 정보 스크래핑 실행**

이제 `get_book_urls_from_category` 함수로 모든 카테고리에서 책 URL들을 수집하고 (또는 선택한 몇 개 카테고리에서만), 각 URL에 대해 `scrape_book_details` 함수를 호출하여 전체 책 데이터를 만듭니다.
실제 운영 시에는 매우 많은 요청이 발생할 수 있으므로, 여기서는 **일부 카테고리**와 각 카테고리당 **제한된 페이지 수**, 그리고 **전체 책 수 제한**을 두어 실습합니다.

In [ ]:
final_book_data_list = []
all_target_book_urls = set() # 최종적으로 스크랩할 책 URL들 (중복 제거)

# 수집 대상 카테고리 수 제한 (예: 처음 2개 카테고리만)
categories_to_scrape = all_categories[:2] if all_categories else [] # slice, e.g. :2
MAX_BOOKS_TO_SCRAPE_TOTAL = 20 # 전체 스크래핑할 책의 최대 개수 (테스트용, e.g. 10)

In [ ]:
categories_to_scrape

[{'name': 'Travel',
  'url': 'http://books.toscrape.com/catalogue/category/books/travel_2/index.html'},
 {'name': 'Mystery',
  'url': 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html'}]

In [ ]:
print(f"\n총 {len(categories_to_scrape)}개 카테고리에서 책 URL 수집 시작...")
for category in categories_to_scrape:
    print(f"'{category['name']}' 카테고리 처리 중...")
    # 각 카테고리당 최대 1 페이지만 (시간 절약 위해)
    book_urls = get_book_urls_from_category(category['url'], max_pages_per_category=1) # get_book_urls_from_category, page limit (e.g. 1)
    for url in book_urls:
        all_target_book_urls.add(url)
        if len(all_target_book_urls) >= MAX_BOOKS_TO_SCRAPE_TOTAL: # MAX_BOOKS_TO_SCRAPE_TOTAL
            break # 전체 책 수 제한 도달 시 URL 수집 중단
    if len(all_target_book_urls) >= MAX_BOOKS_TO_SCRAPE_TOTAL: # MAX_BOOKS_TO_SCRAPE_TOTAL
        print("전체 스크래핑할 책의 최대 개수에 도달했습니다.")
        break


총 2개 카테고리에서 책 URL 수집 시작...
'Travel' 카테고리 처리 중...
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/category/books/travel_2/index.html
http://books.toscrape.com/catalogue/category/books/travel_2/index.html 카테고리의 마지막 페이지에 도달했거나, 다음 페이지 버튼이 없습니다!
'Mystery' 카테고리 처리 중...
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/category/books/mystery_3/index.html
전체 스크래핑할 책의 최대 개수에 도달했습니다.


In [ ]:
print(f"\n총 {len(all_target_book_urls)}개의 책 상세 정보를 스크래핑합니다.")
for i, book_url in enumerate(list(all_target_book_urls)):
    print(f"--- {i+1}/{len(all_target_book_urls)}번째 책 스크래핑 시작 --- ")
    book_info = scrape_book_details(book_url) # scrape_book_details
    if book_info:
        final_book_data_list.append(book_info)


총 20개의 책 상세 정보를 스크래핑합니다.
--- 1/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html
--- 2/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/full-moon-over-noahs-ark-an-odyssey-to-mount-ararat-and-beyond_811/index.html
--- 3/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/hide-away-eve-duncan-20_620/index.html
--- 4/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/a-time-of-torment-charlie-parker-14_657/index.html
--- 5/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/the-murder-of-roger-ackroyd-hercule-poirot-4_852/index.html
--- 6/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/murder-at-the-42nd-street-library-raymond-ambler-1_624/index.html
--- 7/20번째 책 스크래핑 시작 --- 
페이지 요청 중 (1번째 시도): http://books.toscrape.com/catalogue/a-murder-in-time_877/index.html
--- 8/20번째 책 스크래핑 시작 --- 
페이지

In [ ]:
print(f"\n총 {len(final_book_data_list)}개의 책 상세 정보 수집 완료.")
if final_book_data_list:
    print("수집된 첫 번째 책 정보:")
    print(final_book_data_list[0])


총 20개의 책 상세 정보 수집 완료.
수집된 첫 번째 책 정보:
{'url': 'http://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html', 'title': 'A Study in Scarlet (Sherlock Holmes #1)', 'price_raw': '£16.73', 'price': 16.73, 'stock_availability_raw': 'In stock (14 available)', 'stock_available_count': 14, 'star_rating': 2, 'description': "In the debut of literature's most famous sleuth, a dead man is discovered in a bloodstained room in Brixton. The only clues are a wedding ring, a gold watch, a pocket edition of Boccaccio's Decameron, and a word scrawled in blood on the wall. With this investigation begins the partnership of Sherlock Holmes and Dr. Watson. Their search for the murderer uncovers a story of In the debut of literature's most famous sleuth, a dead man is discovered in a bloodstained room in Brixton. The only clues are a wedding ring, a gold watch, a pocket edition of Boccaccio's Decameron, and a word scrawled in blood on the wall. With this investigation begins the par

## 💾 4단계: 데이터 통합 및 저장

수집된 모든 책 정보를 Pandas DataFrame으로 변환하여 분석 및 저장이 용이하도록 합니다.

**4.1. Pandas DataFrame으로 변환**

리스트 형태로 저장된 책 데이터(`final_book_data_list`)를 DataFrame으로 만듭니다.

In [ ]:
if final_book_data_list:
    df_books = pd.DataFrame(final_book_data_list) # DataFrame
    print("\n--- Pandas DataFrame 변환 결과 (상위 5개) ---")
    #pd.set_option('display.max_columns', None) # 'display.max_columns'
    #pd.set_option('display.width', 1000)      # 'display.width', 1000
    print(df_books.head()) # head
    print("\n--- DataFrame 정보 ---")
    df_books.info() #

In [ ]:
print("\n--- 기술 통계 (숫자형 데이터) ---")
print(df_books.describe(include='number')) # describe, 'number'

In [ ]:
if not final_book_data_list:
    print("\n수집된 데이터가 없어 DataFrame으로 변환할 수 없습니다.")
    df_books = pd.DataFrame() # 빈 DataFrame 생성

**4.2. CSV 파일로 저장**

생성된 DataFrame을 CSV 파일로 저장하여 나중에 사용하거나 다른 도구로 분석할 수 있도록 합니다.

In [ ]:
if not df_books.empty:
    csv_filename = 'books_scraped_advanced.csv' # 'books_scraped_advanced.csv'
    try:
        df_books.to_csv(csv_filename, index=False, encoding='utf-8-sig') # to_csv, csv_filename, False, 'utf-8-sig'
        print(f"\n'{csv_filename}' 파일로 저장 완료!")
        print("Colab 환경의 좌측 '파일' 탭에서 해당 파일을 확인하고 다운로드할 수 있습니다.")
    except Exception as e:
        print(f"CSV 파일 저장 중 오류 발생: {e}")
else:
    print("\n저장할 데이터가 없습니다.")



'books_scraped_advanced.csv' 파일로 저장 완료!
Colab 환경의 좌측 '파일' 탭에서 해당 파일을 확인하고 다운로드할 수 있습니다.
